In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import pandas as pd
import time
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re

In [13]:
driver = uc.Chrome()

In [14]:
df = pd.read_csv('atasan_pria.csv')

In [15]:
df['Link'][0]

'https://ta.tokopedia.com/promo/v1/clicks/8a-xgVY2gmUEHsJpHAHdHsrpbm-xgVY789CBUsthbm-Orfua9fBjUstiHmUDUSoBrRo2QfNwq3ei6i-CQ_VFzsCuZfHpoB2de1ga8cxO1juq_OV5oOzA_IPizjWiuVtd31OAqB4N33H7e9-NeMzFPjUaejNU9foG8BHh1JBC1VoRP_HWzJBGz1hkoMPcujtFZ7VV33OOqurpu3xx1SViyfzvqczC_9JOH72v1AC68jydoJPpgVuzySPuPZUDUSz2Q31i6sJRHsUpHsKFosrDUMoFyaUEHp1OHmFircYpq9z2Qfdi6sJDUMVDgaUEUMoxPcuSQR-N9f-N9RCaQfzOyReibm-XP3Oig9-wQfgwy3zpUsthoZFiQSuWyMua9fVjrOYag9Ji6sJObm-sy9zwq3zpUs2QH_rO6ZFhopK7bAHOopBPbm-X9foxQMz2gcV7guYxgIHi6sJ7HmFiy3-wPcupPmUEUjdibm-FQRo79fVDgaUEUMzBgiUDUSgBrSo2Qfdi6i-fHiUDUMVj9RosQR-BUstpopjXosnFoA1hH_K7o_rao_HDUSHp9fh5gaUEUMuNZM2jZJ2M33NGPMep_Mh-qMY2_1H7oJNJ__CvzVjF_uzCHJY13_ooucD7Z3BRqujp1SByH7ND3uxGqMVAZ_g-qjuO_7zVoJO1gRxvzJ17_uzS8JYJz_zo17BpZ3O7QcuygIgsQu-Myp-6PMoWu3Bvq1BRZ3BRq3-W69ugHBu2_fBGqJN9Z_VqzOg23MjF8JOk_M2HuV-0_OzuPJYJZ_C68_Cd3_--HBBEgRx68Mx2392zq1hAZSgsQ3hXyurOgB2IuSPyHMh0Z325q1OAZ9o-Q_BNyuPjrc-D69PsQ_B0gVP6HVKaQcW-qMY2_1o-r7BW69BxufzFyMFN8MVI69PyHMh0Z325q1OAZ9o-Qjyh3BxGouKp1MxqH7O2_f

In [16]:
dct_data = {
    "nama_toko": [],
    "daerah_toko": [],
    "jumlah_terjual": [],
    "stok_tersisa": []
}

In [17]:
def stock_finder(p_stok, stok_total):
    res_stok = p_stok.find_elements(By.CSS_SELECTOR, 'b')[0].text
    res_stok = re.sub(r'\D', '', res_stok)
    if res_stok == '':
        res_stok = 0
    else:
        res_stok = int(res_stok)
    print(res_stok)
    stok_total += res_stok

    return stok_total

In [18]:
for link in df['Link']:
    #Try if the web is still not ready
    try_time = 5
    for i in range(try_time):
        try:    
            driver.get(link)
            time.sleep(4)

            # Nama Toko data
            h2_nama_toko = driver.find_element(By.XPATH, '//h2[@class="css-1wdzqxj-unf-heading e1qvo2ff2"]')
            break
        except:
            try_time -= 1
    if try_time == 0:
        print(f'Error loading page after 5 try time')
        break
    
    dct_data['nama_toko'].append(h2_nama_toko.text)

    # Daerah Data
    h2_daerah = driver.find_element(By.XPATH, '//h2[@class="css-1pd07ge-unf-heading e1qvo2ff2"]')
    h2_daerah.find_elements(By.CSS_SELECTOR, 'b')[0].text

    dct_data['daerah_toko'].append(h2_nama_toko.text)

    # Terjual Data
    try:
        p_sold = driver.find_element(By.XPATH, '//p[@data-testid="lblPDPDetailProductSoldCounter"]')

        sold_val = int(re.sub(r'\D', '', p_sold.text)) 
    except:
        sold_val = 0

    dct_data['jumlah_terjual'].append(sold_val)
    
    # Stok Data
    try:
        div_variant = driver.find_element(By.XPATH, '//div[@class="css-1b2d3hk"]')
        parent_level_div = div_variant.find_elements(By.CSS_SELECTOR, 'div')
        all_level_div = [x for x in parent_level_div if x.get_attribute('class')=='css-hayuji']

        len_level1 = len(all_level_div[0].find_elements(By.CSS_SELECTOR, 'button'))

        stok_total = 0
        for el_level1 in all_level_div[0].find_elements(By.CSS_SELECTOR, 'button'):
            el_level1.click()

            if len(all_level_div) >= 2:
                for el_level2 in all_level_div[1].find_elements(By.CSS_SELECTOR, 'button'):
                    el_level2.click()
                    p_stok = driver.find_element(By.XPATH, '//p[@class="css-1yy88m3-unf-heading e1qvo2ff8"]')
                    stok_total = stock_finder(p_stok, stok_total)

            else:
                p_stok = driver.find_element(By.XPATH, '//p[@class="css-1yy88m3-unf-heading e1qvo2ff8"]')
                stok_total = stock_finder(p_stok, stok_total)
    except:
        p_stok = driver.find_element(By.XPATH, '//p[@class="css-1yy88m3-unf-heading e1qvo2ff8"]')
        stok_total = stock_finder(p_stok, stok_total)
    

    dct_data['stok_tersisa'].append(stok_total)
    

Error loading page after 5 try time


In [39]:
driver.get("https://www.tokopedia.com/optikjuni/frame-kacamata-kayu-pria-ringan-minus-anti-radiasi-fashion-polarize-46-satu-warna-blue-ray?extParam=ivf%3Dfalse")
# driver.get("https://www.tokopedia.com/liquorfashion/kacamata-bridges-kacamata-hitam-wanita-severn-s-bi-bv-severn-45-55?extParam=ivf%3Dfalse")


In [40]:
try:
    p_sold = driver.find_element(By.XPATH, '//p[@data-testid="lblPDPDetailProductSoldCounter"]')
    import re

    cleaned_text = re.sub(r'\D', '', p_sold.text)  # \D matches any non-digit character
    print(int(cleaned_text))
except:
    print('Sold not found')
    cleaned_text = 0
    print(cleaned_text)


750


In [41]:
h2_nama_toko = driver.find_element(By.XPATH, '//h2[@class="css-1wdzqxj-unf-heading e1qvo2ff2"]')

In [42]:
h2_nama_toko.text

'Optik Juni'

In [43]:
h2_daerah = driver.find_element(By.XPATH, '//h2[@class="css-1pd07ge-unf-heading e1qvo2ff2"]')

h2_daerah.find_elements(By.CSS_SELECTOR, 'b')[0].text

'Jakarta Pusat'

In [44]:
p_stok = driver.find_element(By.XPATH, '//p[@class="css-1yy88m3-unf-heading e1qvo2ff8"]')


In [47]:
res_stok = p_stok.find_elements(By.CSS_SELECTOR, 'b')[0].text
res_stok = re.sub(r'\D', '', res_stok)
if res_stok == '':
    res_stok = 0
else:
    res_stok = int(res_stok)
res_stok

499

In [69]:
#IF
div_variant = driver.find_element(By.XPATH, '//div[@class="css-1b2d3hk"]')

In [73]:
parent_level_div = div_variant.find_elements(By.CSS_SELECTOR, 'div')
all_level_div = [x for x in parent_level_div if x.get_attribute('class')=='css-hayuji']

In [75]:
len(all_level_div)

2

In [83]:
def stock_finder(p_stok, stok_total):
    res_stok = p_stok.find_elements(By.CSS_SELECTOR, 'b')[0].text
    res_stok = re.sub(r'\D', '', res_stok)
    if res_stok == '':
        res_stok = 0
    else:
        res_stok = int(res_stok)
    print(res_stok)
    stok_total += res_stok

    return stok_total

In [84]:
len_level1 = len(all_level_div[0].find_elements(By.CSS_SELECTOR, 'button'))

stok_total = 0
for el_level1 in all_level_div[0].find_elements(By.CSS_SELECTOR, 'button'):
    el_level1.click()

    if len(all_level_div) >= 2:
        for el_level2 in all_level_div[1].find_elements(By.CSS_SELECTOR, 'button'):
            el_level2.click()
            stok_total = stock_finder(p_stok, stok_total)

    else:
        res_stok = p_stok.find_elements(By.CSS_SELECTOR, 'b')[0].text
        res_stok = re.sub(r'\D', '', res_stok)
        if res_stok == '':
            res_stok = 0
        else:
            res_stok = int(res_stok)
        print(res_stok)
        stok_total = stock_finder(p_stok, stok_total)

            

493
497
496
497
499
498
500
500
20
0
0
0
0
0
0
0
0
0


In [85]:
stok_total

4000